#  DataBase contruction

In [25]:
import pandas as pd
import itertools
from itertools import combinations
# import sns
from matplotlib import pyplot as plt
from scipy.stats import chi2_contingency

In [26]:
# reading all the data files
data_clinical_patient = pd.read_csv('pan_origimed_2020/data_clinical_patient.txt', sep="\t")
data_clinical_sample = pd.read_csv('pan_origimed_2020/data_clinical_sample.txt', sep="\t")
data_cna_log2 = pd.read_csv('pan_origimed_2020/data_cna_log2.txt', sep="\t")
data_cna = pd.read_csv('pan_origimed_2020/data_cna.txt', sep="\t")
data_mutations = pd.read_csv('pan_origimed_2020/data_mutations.txt', sep="\t", header=2, dtype={"Exon_Number": "string"})
data_sv = pd.read_csv('pan_origimed_2020/data_sv.txt', sep="\t")

In [27]:
# removing bad rows
data_clinical_sample = data_clinical_sample[4:]
data_clinical_patient = data_clinical_patient[4:]

In [28]:
data_clinical_patient = data_clinical_patient.rename(columns={'#Patient Identifier': 'PATIENT_ID'})

In [29]:
data_clinical_patient.head()

,PATIENT_ID,Sex,Diagnosis Age,Smoke Status,Treatment
4,Patient0001,Female,67,Unknown,Other_Treatments
5,Patient0002,Male,75,Unknown,Treatment-naive
6,Patient0003,Female,45,Unknown,Treatment-naive
7,Patient0004,Male,70,Unknown,Treatment-naive
8,Patient0005,Male,53,Unknown,Treatment-naive


In [30]:
# matching the sample id to match other tables
data_clinical_patient["SAMPLE_ID"] = data_clinical_patient["PATIENT_ID"].apply(lambda x: "P-" + x[7:])

In [31]:
# make all sample id header name the same - "SAMPLE_ID"
data_clinical_sample.rename(columns={"Sample Identifier": 'SAMPLE_ID'}, inplace=True)
data_mutations.rename(columns={"Tumor_Sample_Barcode": 'SAMPLE_ID'}, inplace=True)
data_sv.rename(columns={"Sample_Id": 'SAMPLE_ID'}, inplace=True)

In [32]:
# merge everything
merged_clinical_data = data_clinical_patient.merge(data_clinical_sample, on="SAMPLE_ID", how='outer')
merged_mutations_data = merged_clinical_data.merge(data_mutations, on="SAMPLE_ID", how='outer')
merged_all_data = merged_mutations_data.merge(data_sv, on="SAMPLE_ID", how='outer')

In [33]:
merged_all_data["SNP_event"] = merged_all_data["Reference_Allele"].fillna("").astype(str) + ">" + merged_all_data["Tumor_Seq_Allele2"].fillna("").astype(str)


In [34]:
data_for_model = merged_all_data[["PATIENT_ID", "Cancer Type", 'Cancer Type Detailed', 'Tumor Stage',
                                'Sample Type', "Sex", "Diagnosis Age", "Smoke Status", "TMB (nonsynonymous)",
                                "Hugo_Symbol", "Chromosome", "Start_Position", "End_Position",
                                "Consequence", "Variant_Type", "SNP_event", "Protein_position", "Codons",
                                "Exon_Number","VAR_TYPE_SX", "Site1_Hugo_Symbol", "Site2_Hugo_Symbol","Event_Info"]]

In [35]:
data_for_model.shape

(105906, 23)

In [36]:
data_for_model.head(20)

,PATIENT_ID,Cancer Type,Cancer Type Detailed,Tumor Stage,Sample Type,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,...,Consequence,Variant_Type,SNP_event,Protein_position,Codons,Exon_Number,VAR_TYPE_SX,Site1_Hugo_Symbol,Site2_Hugo_Symbol,Event_Info
0,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,KMT2C,...,missense_variant,SNP,C>T,4822.0,cGt/cAt,57/59,Substitution/Indel,NaN,NaN,NaN
1,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,KRAS,...,missense_variant,SNP,C>A,12.0,Ggt/Tgt,5-Feb,Substitution/Indel,NaN,NaN,NaN
2,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,SOX9,...,frameshift_variant,DEL,C>-,236.0,aCc/ac,3-Mar,Truncation,NaN,NaN,NaN
3,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,APC,...,"stop_gained,splice_region_variant",SNP,C>T,216.0,Cga/Tga,16-Jul,Truncation,NaN,NaN,NaN
4,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,APC,...,stop_gained,SNP,G>T,1286.0,Gaa/Taa,16/16,Truncation,NaN,NaN,NaN
5,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,QKI,...,missense_variant,SNP,G>A,47.0,Gaa/Aaa,8-Jan,Substitution/Indel,NaN,NaN,NaN
6,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,FBXW7,...,missense_variant,SNP,G>A,505.0,Cgc/Tgc,12-Oct,Substitution/Indel,NaN,NaN,NaN
7,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,PTPN11,...,intron_variant,DEL,TTTC>-,NaN,NaN,<NA>,Substitution/Indel,NaN,NaN,NaN
8,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,GLI1,...,missense_variant,SNP,G>T,488.0,Gct/Tct,12-Nov,Substitution/Indel,NaN,NaN,NaN
9,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,ATM,...,"splice_region_variant,intron_variant",SNP,G>A,1026.0,NaN,<NA>,Substitution/Indel,NaN,NaN,NaN


In [37]:
# Function to handle the conversion
def convert_exon_number(val):
    try:
        # First, try to convert to 'Month-Year' format (e.g., 'Sep-89' -> '09/89')
        return pd.to_datetime(val, format='%b-%y').strftime('%m/%y')
    except ValueError:
        pass

    try:
        # Then, try to convert to 'DD-Mon' format (e.g., '14-Sep' -> '09/14')
        date_obj = pd.to_datetime(val, format='%d-%b', errors='raise')
        return date_obj.strftime('%m/%d')
    except ValueError:
        # If neither format matches, return the value as is (non-date-like string)
        return val

In [38]:
# Apply the function to the column
data_for_model.loc[:, 'Exon_Number'] = data_for_model['Exon_Number'].apply(convert_exon_number)

In [39]:
data_for_model.head(10)

,PATIENT_ID,Cancer Type,Cancer Type Detailed,Tumor Stage,Sample Type,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,...,Consequence,Variant_Type,SNP_event,Protein_position,Codons,Exon_Number,VAR_TYPE_SX,Site1_Hugo_Symbol,Site2_Hugo_Symbol,Event_Info
0,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,KMT2C,...,missense_variant,SNP,C>T,4822.0,cGt/cAt,57/59,Substitution/Indel,NaN,NaN,NaN
1,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,KRAS,...,missense_variant,SNP,C>A,12.0,Ggt/Tgt,02/05,Substitution/Indel,NaN,NaN,NaN
2,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,SOX9,...,frameshift_variant,DEL,C>-,236.0,aCc/ac,03/03,Truncation,NaN,NaN,NaN
3,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,APC,...,"stop_gained,splice_region_variant",SNP,C>T,216.0,Cga/Tga,07/16,Truncation,NaN,NaN,NaN
4,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,APC,...,stop_gained,SNP,G>T,1286.0,Gaa/Taa,16/16,Truncation,NaN,NaN,NaN
5,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,QKI,...,missense_variant,SNP,G>A,47.0,Gaa/Aaa,01/08,Substitution/Indel,NaN,NaN,NaN
6,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,FBXW7,...,missense_variant,SNP,G>A,505.0,Cgc/Tgc,10/12,Substitution/Indel,NaN,NaN,NaN
7,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,PTPN11,...,intron_variant,DEL,TTTC>-,NaN,NaN,<NA>,Substitution/Indel,NaN,NaN,NaN
8,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,GLI1,...,missense_variant,SNP,G>T,488.0,Gct/Tct,11/12,Substitution/Indel,NaN,NaN,NaN
9,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,ATM,...,"splice_region_variant,intron_variant",SNP,G>A,1026.0,NaN,<NA>,Substitution/Indel,NaN,NaN,NaN


In [40]:
data_for_model["Cancer Type"].value_counts()

Cancer Type
Colorectal Carcinoma                     28396
Non Small Cell Lung Cancer               19526
Gastric Cancer                           10832
Liver Hepatocellular Carcinoma            9633
Esophageal Carcinoma                      6298
Intrahepatic Cholangiocarcinoma           3939
Small Cell Lung Cancer                    3025
Pancreatic Cancer                         2957
Extrahepatic Cholangiocarcinoma           2883
Breast Carcinoma                          2350
Soft Tissue Sarcoma                       2270
Gallbladder Carcinoma                     2192
Ovarian Carcinoma                         1851
Urothelial Carcinoma                      1631
Kidney Renal Cell Carcinoma               1500
Uterine Corpus Endometrial Carcinoma      1358
Cancer of Unknown Primary                 1068
Head and Neck Carcinoma                    997
Carcinoma of Uterine Cervix                787
Small Bowel Carcinoma                      755
Bone Sarcoma                               601
G

In [51]:
data_for_model["VAR_TYPE_SX"].value_counts()
data_for_model.shape

(80153, 66)

In [19]:
def create_age_range(x):
    if x <= 10:
        return "0-10"
    elif x <= 20:
        return "11-20"
    elif x <= 30:
        return "21-30"
    elif x <= 40:
        return "31-40"
    elif x <= 50:
        return "41-50"
    elif x <= 60:
        return "51-60"
    elif x <= 70:
        return "61-70"
    elif x <= 80:
        return "71-80"
    else:
        return "80+"

In [20]:
data_for_model.loc[:, 'Diagnosis Age'] = data_for_model['Diagnosis Age'].astype(int).apply(create_age_range).astype("category")

In [42]:
# Filter cancer types with at least 2000 samples
cancer_counts = data_for_model['Cancer Type'].value_counts()
valid_cancer_types = cancer_counts[cancer_counts >= 2000].index
data_for_model = data_for_model[data_for_model['Cancer Type'].isin(valid_cancer_types)]
data_for_model = data_for_model[data_for_model['Chromosome'].notnull()]

In [416]:
data_for_lift = data_for_model.copy()

,PATIENT_ID,Cancer Type,Cancer Type Detailed,Tumor Stage,Sample Type,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,...,Consequence,Variant_Type,SNP_event,Protein_position,Codons,Exon_Number,VAR_TYPE_SX,Site1_Hugo_Symbol,Site2_Hugo_Symbol,Event_Info
0,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,61-70,Unknown,0.333333333,KMT2C,...,missense_variant,SNP,C>T,4822.0,cGt/cAt,57/59,Substitution/Indel,NaN,NaN,NaN
1,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,61-70,Unknown,0.333333333,KRAS,...,missense_variant,SNP,C>A,12.0,Ggt/Tgt,02/05,Substitution/Indel,NaN,NaN,NaN
2,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,61-70,Unknown,0.333333333,SOX9,...,frameshift_variant,DEL,C>-,236.0,aCc/ac,03/03,Truncation,NaN,NaN,NaN
3,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,61-70,Unknown,0.333333333,APC,...,"stop_gained,splice_region_variant",SNP,C>T,216.0,Cga/Tga,07/16,Truncation,NaN,NaN,NaN
4,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,61-70,Unknown,0.333333333,APC,...,stop_gained,SNP,G>T,1286.0,Gaa/Taa,16/16,Truncation,NaN,NaN,NaN


In [43]:
data_for_model['Consequence'].str.split(',')
dummy_vars = data_for_model['Consequence'].str.split(',').explode().str.get_dummies().groupby(level=0).sum()
data_for_model = data_for_model.join(dummy_vars)
data_for_model.drop('Consequence', axis=1, inplace=True)

In [44]:
# Create dummy variables for the Chromosome column ONLY FOR RULES!
chromosome_dummy = data_for_model['Chromosome'].str.get_dummies()

# Rename columns to have 'chr_' prefix
chromosome_dummy.columns = [f'chr_{col}' for col in chromosome_dummy.columns]

# Join the dummy variables to the original dataframe
data_for_model = data_for_model.join(chromosome_dummy)

# Drop the original Chromosome column
data_for_model.drop('Chromosome', axis=1, inplace=True)

In [46]:
# Drop columns and rows with NAs ONLY FOR RULES!
data_for_model = data_for_model.drop(['Site1_Hugo_Symbol', 'Site2_Hugo_Symbol', 'Event_Info'], axis=1).dropna()

In [48]:
chromosome_dummy.columns

Index(['chr_1', 'chr_10', 'chr_11', 'chr_12', 'chr_13', 'chr_14', 'chr_15',
       'chr_16', 'chr_17', 'chr_18', 'chr_19', 'chr_2', 'chr_20', 'chr_21',
       'chr_22', 'chr_3', 'chr_4', 'chr_5', 'chr_6', 'chr_7', 'chr_8', 'chr_9',
       'chr_X'],
      dtype='object')

In [52]:
data_for_model.head(20)

,PATIENT_ID,Cancer Type,Cancer Type Detailed,Tumor Stage,Sample Type,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,...,chr_21,chr_22,chr_3,chr_4,chr_5,chr_6,chr_7,chr_8,chr_9,chr_X
0,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,KMT2C,...,0,0,0,0,0,0,1,0,0,0
1,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,KRAS,...,0,0,0,0,0,0,0,0,0,0
2,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,SOX9,...,0,0,0,0,0,0,0,0,0,0
3,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,APC,...,0,0,0,0,1,0,0,0,0,0
4,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,APC,...,0,0,0,0,1,0,0,0,0,0
5,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,QKI,...,0,0,0,0,0,1,0,0,0,0
6,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,FBXW7,...,0,0,0,1,0,0,0,0,0,0
8,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,67,Unknown,0.333333333,GLI1,...,0,0,0,0,0,0,0,0,0,0
10,Patient0002,Colorectal Carcinoma,Colorectal Adenocarcinoma,II,Primary,Male,75,Unknown,0.3,SOX9,...,0,0,0,0,0,0,0,0,0,0
11,Patient0002,Colorectal Carcinoma,Colorectal Adenocarcinoma,II,Primary,Male,75,Unknown,0.3,EPHB1,...,0,0,1,0,0,0,0,0,0,0


In [49]:
data_for_model.to_csv("data_for_rules.csv", index=False)

In [380]:
hypo_data = pd.read_csv("hypotheses.csv")

In [381]:
hypo_data_updates = hypo_data[hypo_data["support"] > 2].sort_values(["cancer_type", 'support'], ascending=[True, False])
hypo_data_updates.head()

,cancer_type,Smoke Status,Hugo_Symbol,TMB (nonsynonymous),Start_Position,Protein_position,Sex,SNP_event,Site1_Hugo_Symbol,Diagnosis Age,...,upstream_gene_variant,Site2_Hugo_Symbol,Event_Info,Exon_Number,splice_region_variant,End_Position,5_prime_UTR_variant,missense_variant,splice_donor_variant,support
137,Bone Sarcoma,NaN,NaN,0.033333,NaN,NaN,NaN,NaN,NaN,11-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
174,Bone Sarcoma,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,11-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
190,Bone Sarcoma,NaN,NaN,0.200000,NaN,NaN,NaN,NaN,NaN,11-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
274,Bone Sarcoma,NaN,NaN,0.100000,NaN,NaN,NaN,NaN,NaN,11-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
112,Breast Carcinoma,NaN,NaN,NaN,NaN,NaN,Female,NaN,NaN,41-50,...,NaN,NCOR1,NCOR1-Intragenic,NaN,NaN,NaN,NaN,NaN,NaN,11


In [382]:
def cancer_type_correlations(df):
    """
    Print cancer type and non-null feature-value pairs for each row in the DataFrame.

    Parameters:
    - df: DataFrame with columns "cancer_type", feature columns, and "support".
    """
    corr_list = []
    # Iterate through each row
    for index, row in df.iterrows():
        # Extract cancer type and support
        cancer_type = row["cancer_type"]
        support = row["support"]

        # Get feature-value pairs where the feature value is not null
        features = [
            f"{feature}={row[feature]}"
            for feature in df.columns
            if feature not in {"cancer_type", "support"} and not pd.isnull(row[feature])
        ]

        # Format and print the result
        features_str = ", ".join(features)
        corr_list.append(f"{cancer_type}: {features_str}, Support: {support}")
    return corr_list


In [383]:
corr_list = cancer_type_correlations(hypo_data_updates)

In [384]:
corr_list

['Bone Sarcoma: TMB (nonsynonymous)=0.033333333, Diagnosis Age=11-20, Support: 8',
 'Bone Sarcoma: TMB (nonsynonymous)=0.0, Diagnosis Age=11-20, Support: 6',
 'Bone Sarcoma: TMB (nonsynonymous)=0.2, Diagnosis Age=11-20, Support: 6',
 'Bone Sarcoma: TMB (nonsynonymous)=0.1, Diagnosis Age=11-20, Support: 3',
 'Breast Carcinoma: Sex=Female, Diagnosis Age=41-50, Site2_Hugo_Symbol=NCOR1, Event_Info=NCOR1-Intragenic, Support: 11',
 'Breast Carcinoma: TMB (nonsynonymous)=0.333333333, Sex=Female, Site1_Hugo_Symbol=BRCA2, Site2_Hugo_Symbol=BRCA2, Event_Info=BRCA2-BRCA2, Support: 7',
 'Breast Carcinoma: Hugo_Symbol=PIK3CA, Start_Position=178952085.0, Sex=Female, Exon_Number=21/21, End_Position=178952085.0, Support: 6',
 'Breast Carcinoma: Sex=Female, Site1_Hugo_Symbol=BRCA1, Diagnosis Age=31-40, Site2_Hugo_Symbol=BRCA1, Event_Info=BRCA1-Intragenic, Support: 4',
 'Breast Carcinoma: TMB (nonsynonymous)=0.766666667, Sex=Female, Diagnosis Age=51-60, Event_Info=LRP1B-Intragenic, Support: 4',
 'Breast

In [420]:
cancer_type_dummy = data_for_lift['Cancer Type'].str.get_dummies().groupby(level=0).sum()
data_for_lift = data_for_lift.join(cancer_type_dummy)

In [422]:
data_for_lift.head()

,PATIENT_ID,Cancer Type,Cancer Type Detailed,Tumor Stage,Sample Type,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,...,Esophageal Carcinoma,Extrahepatic Cholangiocarcinoma,Gallbladder Carcinoma,Gastric Cancer,Intrahepatic Cholangiocarcinoma,Liver Hepatocellular Carcinoma,Non Small Cell Lung Cancer,Pancreatic Cancer,Small Cell Lung Cancer,Soft Tissue Sarcoma
0,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,61-70,Unknown,0.333333333,KMT2C,...,0,0,0,0,0,0,0,0,0,0
1,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,61-70,Unknown,0.333333333,KRAS,...,0,0,0,0,0,0,0,0,0,0
2,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,61-70,Unknown,0.333333333,SOX9,...,0,0,0,0,0,0,0,0,0,0
3,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,61-70,Unknown,0.333333333,APC,...,0,0,0,0,0,0,0,0,0,0
4,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,Primary,Female,61-70,Unknown,0.333333333,APC,...,0,0,0,0,0,0,0,0,0,0


In [424]:
# Combine columns to create specific mutation identifiers
# data_for_lift['Mutation'] = data_for_lift['Chromosome'] + "_" + data_for_lift['Start_Position'].astype(str) + "_" + data_for_lift['Variant_Type']
data_for_lift['Position'] = data_for_lift['Start_Position'].astype(str) + "-" + data_for_lift['End_Position'].astype(str)

In [445]:
# Select a subset of columns to analyze (e.g., most relevant ones)
columns_to_combine = ['Sex', 'Smoke Status', 'Chromosome', 'Hugo_Symbol', 'SNP_event', "Consequence", 'Exon_Number',
                      "Diagnosis Age", "TMB (nonsynonymous)", "Position", "Protein_position", "Codons", "VAR_TYPE_SX",
                      "Site1_Hugo_Symbol", "Site2_Hugo_Symbol", "Event_Info"]

In [392]:
lifts = []
for cancer_type in cancer_type_dummy.columns:
    # Calculate the probability of the cancer type
    P_B = data[cancer_type].mean()

    for feature_pair in feature_combinations:
        # Create a combined feature
        combined_feature = data[feature_pair[0]].astype(str) + "_" + data[feature_pair[1]].astype(str)
        # Filter out combinations with very low counts
        min_count = 100
        P_A_counts = combined_feature.value_counts()

        # Filter combined features based on minimum count
        valid_features = P_A_counts[P_A_counts >= min_count].index
        filtered_data = combined_feature[combined_feature.isin(valid_features)]

        # Recalculate probabilities with filtered data
        P_A = filtered_data.value_counts(normalize=True)
        P_A_and_B = filtered_data[data[cancer_type] == 1].value_counts(normalize=True)
        lift = (P_A_and_B / (P_A * P_B)).round(2)
        lifts.append((cancer_type, feature_pair, lift))

KeyError: 'Diagnostic Age'

In [339]:
# Flatten the results for easy visualization
lift_results = []

for cancer_type, feature_pair, lift in lifts:
    for feature_value, lift_value in lift.items():
        lift_results.append({
            'Cancer Type': cancer_type,
            'Feature Pair': feature_pair,
            'Feature Value': feature_value,
            'Lift': lift_value
        })

lift_df = pd.DataFrame(lift_results)
lift_df = lift_df.sort_values(by='Lift', ascending=False)

In [340]:
lift_df.head(20)

,Cancer Type,Feature Pair,Feature Value,Lift
410,Breast Carcinoma,"(Sex, Exon_Number)",Female_21/21,1020.37
19610,Small Cell Lung Cancer,"(Smoke Status, Hugo_Symbol)",Smoker_RB1,943.76
96,Breast Carcinoma,"(Sex, Hugo_Symbol)",Female_MAP3K1,868.14
104,Breast Carcinoma,"(Sex, Hugo_Symbol)",Female_PIK3CA,756.84
1842,Breast Carcinoma,"(SNP_event, Exon_Number)",A>G_21/21,620.73
21627,Soft Tissue Sarcoma,"(Smoke Status, Hugo_Symbol)",Unknown_KIT,551.69
707,Breast Carcinoma,"(Smoke Status, Hugo_Symbol)",Unknown_GATA3,546.63
21117,Soft Tissue Sarcoma,"(Sex, Hugo_Symbol)",Male_KIT,544.15
22142,Soft Tissue Sarcoma,"(Chromosome, Hugo_Symbol)",4_KIT,531.01
1081,Breast Carcinoma,"(Chromosome, Hugo_Symbol)",10_GATA3,525.40


In [341]:
# f = data[data["Chromosome"] == "nan"]
# f = data[data["Chromosome"].isna()]
# f = f[f["Cancer Type"] == "Soft Tissue Sarcoma"]
f = data[data["Exon_Number"] == "21/21"]
# f = f[f["Smoke Status"] == "UnKnown"]
f = f[f["Sex"] == "Female"]
f.head(100)

,PATIENT_ID,Cancer Type,Cancer Type Detailed,Tumor Stage,Sample Type,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,...,Extrahepatic Cholangiocarcinoma,Gallbladder Carcinoma,Gastric Cancer,Intrahepatic Cholangiocarcinoma,Liver Hepatocellular Carcinoma,Non Small Cell Lung Cancer,Pancreatic Cancer,Small Cell Lung Cancer,Soft Tissue Sarcoma,Mutation
664,Patient0041,Breast Carcinoma,Breast Invasive Carcinoma,Unknown,Primary,Female,41-50,Unknown,0.033333333,PIK3CA,...,0,0,0,0,0,0,0,0,0,3_178952085.0_SNP
899,Patient0074,Breast Carcinoma,Breast Invasive Carcinoma,Unknown,Primary,Female,41-50,Unknown,0.166666667,PIK3CA,...,0,0,0,0,0,0,0,0,0,3_178952085.0_SNP
1048,Patient0093,Non Small Cell Lung Cancer,Lung Adenocarcinoma,II,Primary,Female,61-70,Unknown,0.566666667,SETD2,...,0,0,0,0,0,1,0,0,0,3_47058694.0_DEL
1222,Patient0114,Breast Carcinoma,Breast Invasive Carcinoma,IV,Primary,Female,61-70,Unknown,0.1,PIK3CA,...,0,0,0,0,0,0,0,0,0,3_178952085.0_SNP
1751,Patient0160,Breast Carcinoma,Breast Invasive Carcinoma,Unknown,Metastasis,Female,41-50,Unknown,0.233333333,PIK3CA,...,0,0,0,0,0,0,0,0,0,3_178952085.0_SNP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64946,Patient6042,Breast Carcinoma,Breast Invasive Carcinoma,III,Primary,Female,41-50,Unknown,0.433333333,PIK3CA,...,0,0,0,0,0,0,0,0,0,3_178952085.0_SNP
64947,Patient6042,Breast Carcinoma,Breast Invasive Carcinoma,III,Primary,Female,41-50,Unknown,0.433333333,PIK3CA,...,0,0,0,0,0,0,0,0,0,3_178952085.0_SNP
66815,Patient6226,Breast Carcinoma,Breast Invasive Carcinoma,III,Primary,Female,51-60,Unknown,0.2,PIK3CA,...,0,0,0,0,0,0,0,0,0,3_178952085.0_SNP
66816,Patient6226,Breast Carcinoma,Breast Invasive Carcinoma,III,Primary,Female,51-60,Unknown,0.2,PIK3CA,...,0,0,0,0,0,0,0,0,0,3_178952085.0_SNP


In [342]:
feature_combinations_3 = list(combinations(columns_to_combine, 3))

In [ ]:
lifts = []
# Probability of the cancer type
for num_features in range(2, 6):
    feature_combinations = list(combinations(columns_to_combine, num_features))
    for cancer_type in cancer_type_dummy.columns:
        P_B = data_for_lift[cancer_type].mean()
        for feature in feature_combinations:
            # Create a combined feature from three columns
            combined_feature = data_for_lift[feature[0]].astype(str)

            for f in feature[1:]:
                combined_feature += "_" + data_for_lift[f].astype(str)
            # combined_feature = "_".join(data_for_lift[feature].astype(str) for feature in feature_combinations)

            min_count = 100
            P_A_counts = combined_feature.value_counts()

            # Filter combined features based on minimum count
            valid_features = P_A_counts[P_A_counts >= min_count].index
            filtered_data = combined_feature[combined_feature.isin(valid_features)]

            # Probability of the combined feature
            P_A = filtered_data.value_counts(normalize=True)

            # Joint probability of the combined feature and cancer type
            joint = (filtered_data[data_for_lift[cancer_type] == 1].value_counts(normalize=True).reindex(P_A.index, fill_value=0))

            # Calculate lift
            lift = (joint / (P_A * P_B)).round(2)  # Round lift to 2 decimal places for readability

            # Append results as a tuple of the feature triplet and their associated lift values
            lifts.append((cancer_type, feature, lift))

In [347]:
# Flatten the results for easy visualization
lift_results = []

for cancer_type, feature_pair, lift in lifts:
    for feature_value, lift_value in lift.items():
        lift_results.append({
            'Cancer Type': cancer_type,
            'Feature Pair': feature_pair,
            'Feature Value': feature_value,
            'Lift': lift_value
        })

lift_df = pd.DataFrame(lift_results)
lift_df = lift_df.sort_values(by='Lift', ascending=False)

In [348]:
lift_df.sort_values(by='Lift', ascending=False)

,Cancer Type,Feature Pair,Feature Value,Lift
1078,Breast Carcinoma,"(Sex, Chromosome, Exon_Number)",Female_3_21/21,1075.31
1223,Breast Carcinoma,"(Sex, Hugo_Symbol, Exon_Number)",Female_PIK3CA_21/21,1056.89
1287,Breast Carcinoma,"(Sex, SNP_event, Exon_Number)",Female_A>G_21/21,1056.07
436,Breast Carcinoma,"(Sex, Smoke Status, Exon_Number)",Female_Unknown_21/21,1000.34
24665,Small Cell Lung Cancer,"(Smoke Status, Chromosome, Hugo_Symbol)",Smoker_13_RB1,943.76
...,...,...,...,...
27943,Soft Tissue Sarcoma,"(Hugo_Symbol, SNP_event, Exon_Number)",APC_C>T_16/16,NaN
27944,Soft Tissue Sarcoma,"(Hugo_Symbol, SNP_event, Exon_Number)",APC_G>T_16/16,NaN
27947,Soft Tissue Sarcoma,"(Hugo_Symbol, SNP_event, Exon_Number)",EGFR_GGAATTAAGAGAAGC>-_19/28,NaN
27948,Soft Tissue Sarcoma,"(Hugo_Symbol, SNP_event, Exon_Number)",EGFR_T>G_21/28,NaN


In [136]:
filter_triple_data = data[data["Smoke Status"] == "Nonsmoker"]
filter_triple_data = filter_triple_data[filter_triple_data["Hugo_Symbol"] == "TP53"]
filter_triple_data = filter_triple_data[filter_triple_data["SNP_event"] == "G>A"]

In [137]:
filter_triple_data

,PATIENT_ID,Cancer Type,Cancer Type Detailed,Tumor Stage,Sample Type,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,...,splice_region_variant,start_lost,start_retained_variant,stop_gained,stop_lost,stop_retained_variant,synonymous_variant,upstream_gene_variant,Mutation,Is_Lung_Cancer
2475,Patient0210,Non Small Cell Lung Cancer,Lung Adenocarcinoma,IV,Primary,Female,61-70,Nonsmoker,0.2,TP53,...,0,0,0,0,0,0,0,0,17_7578389.0_SNP,1
2854,Patient0258,Non Small Cell Lung Cancer,Lung Adenocarcinoma,I,Primary,Male,51-60,Nonsmoker,0.1,TP53,...,0,0,0,0,0,0,0,0,17_7577539.0_SNP,1
4546,Patient0461,Non Small Cell Lung Cancer,Lung Adenocarcinoma,II,Primary,Female,41-50,Nonsmoker,0.233333333,TP53,...,0,0,0,0,0,0,0,0,17_7574018.0_SNP,1
6611,Patient0645,Non Small Cell Lung Cancer,Lung Adenocarcinoma,I,Primary,Female,71-80,Nonsmoker,0.4,TP53,...,0,0,0,0,0,0,0,0,17_7578517.0_SNP,1
6997,Patient0681,Non Small Cell Lung Cancer,Lung Adenocarcinoma,IV,Primary,Female,51-60,Nonsmoker,0.2,TP53,...,0,0,0,1,0,0,0,0,17_7578524.0_SNP,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88745,Patient8267,Small Cell Lung Cancer,Small Cell Lung Cancer,IV,Primary,Female,41-50,Nonsmoker,1.066666667,TP53,...,0,0,0,0,0,0,0,0,17_7578479.0_SNP,0
89303,Patient8330,Non Small Cell Lung Cancer,Lung Adenocarcinoma,III,Primary,Female,61-70,Nonsmoker,0.2,TP53,...,1,0,0,0,0,0,0,0,17_7576926.0_SNP,1
90975,Patient8492,Small Cell Lung Cancer,Small Cell Lung Cancer,III,Primary,Female,51-60,Nonsmoker,0.266666667,TP53,...,0,0,0,1,0,0,0,0,17_7578275.0_SNP,0
93404,Patient8807,Head and Neck Carcinoma,Head and Neck Squamous Cell Carcinoma,I,Primary,Male,41-50,Nonsmoker,0.1,TP53,...,0,0,0,0,0,0,0,0,17_7577121.0_SNP,0
